# 1. Imported dependencies

In [1]:
#Importing Dependencies
import pandas as pd
import numpy as np
import requests
import json
import time
import datetime
from config import tmdb_key
from pprint import pprint

# 2. Source 1. CSV

In [2]:
ntfl = pd.read_csv('source_data/NetflixOriginals1.csv', encoding = "ISO-8859-1")
ntfl.head(3)

,Title,Genre,Premiere,Runtime,IMDB Score,Language
0,Enter the Anime,Documentary,"August 5, 2019",58,2.5,English/Japanese
1,Dark Forces,Thriller,"August 21, 2020",81,2.6,Spanish
2,The App,Science fiction/Drama,"December 26, 2019",79,2.6,Italian


In [3]:
len(ntfl['Title'])

584

In [4]:
ntfl_df = ntfl[["Title","Genre","Runtime","Language"]]
ntfl_df.head(3)

,Title,Genre,Runtime,Language
0,Enter the Anime,Documentary,58,English/Japanese
1,Dark Forces,Thriller,81,Spanish
2,The App,Science fiction/Drama,79,Italian


In [5]:
# full_dates = net_flix['Premiere']
# converted = pd.to_datetime(full_dates)
# net_flix['Premiere'] = converted
# net_flix.head()

In [6]:
genre_sep_df = ntfl_df.set_index(['Title','Runtime', 'Language']).apply(lambda x: x.str.split('/').explode()).reset_index()      
genre_sep_df.head(5)

,Title,Runtime,Language,Genre
0,Enter the Anime,58,English/Japanese,Documentary
1,Dark Forces,81,Spanish,Thriller
2,The App,79,Italian,Science fiction
3,The App,79,Italian,Drama
4,The Open House,94,English,Horror thriller


In [7]:
lang_sep_df = ntfl_df.set_index(['Title','Runtime','Genre']).apply(lambda x: x.str.split('/').explode()).reset_index()
lang_sep_df.head(3)

,Title,Runtime,Genre,Language
0,Enter the Anime,58,Documentary,English
1,Enter the Anime,58,Documentary,Japanese
2,Dark Forces,81,Thriller,Spanish


In [8]:
csv_df1 = lang_sep_df
csv_df1.head()

,Title,Runtime,Genre,Language
0,Enter the Anime,58,Documentary,English
1,Enter the Anime,58,Documentary,Japanese
2,Dark Forces,81,Thriller,Spanish
3,The App,79,Science fiction/Drama,Italian
4,The Open House,94,Horror thriller,English


In [9]:
csv_df1["actors"] = " "
csv_df1.head(2)

,Title,Runtime,Genre,Language,actors
0,Enter the Anime,58,Documentary,English,
1,Enter the Anime,58,Documentary,Japanese,


In [10]:
csv_df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 610 entries, 0 to 609
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Title     610 non-null    object
 1   Runtime   610 non-null    int64 
 2   Genre     610 non-null    object
 3   Language  610 non-null    object
 4   actors    610 non-null    object
dtypes: int64(1), object(4)
memory usage: 24.0+ KB


In [11]:
csv_df1["actors"].fillna(130, inplace = True)
csv_df1

,Title,Runtime,Genre,Language,actors
0,Enter the Anime,58,Documentary,English,
1,Enter the Anime,58,Documentary,Japanese,
2,Dark Forces,81,Thriller,Spanish,
3,The App,79,Science fiction/Drama,Italian,
4,The Open House,94,Horror thriller,English,
...,...,...,...,...,...
605,Winter on Fire: Ukraine's Fight for Freedom,91,Documentary,Ukranian,
606,Winter on Fire: Ukraine's Fight for Freedom,91,Documentary,Russian,
607,Springsteen on Broadway,153,One-man show,English,
608,Emicida: AmarElo - It's All For Yesterday,89,Documentary,Portuguese,


In [12]:
csv_df1 = csv_df1.rename(columns = {'Title':'title','Runtime':'runtime',"Genre":"genre","Language":"languages"}) 
csv_df1

,title,runtime,genre,languages,actors
0,Enter the Anime,58,Documentary,English,
1,Enter the Anime,58,Documentary,Japanese,
2,Dark Forces,81,Thriller,Spanish,
3,The App,79,Science fiction/Drama,Italian,
4,The Open House,94,Horror thriller,English,
...,...,...,...,...,...
605,Winter on Fire: Ukraine's Fight for Freedom,91,Documentary,Ukranian,
606,Winter on Fire: Ukraine's Fight for Freedom,91,Documentary,Russian,
607,Springsteen on Broadway,153,One-man show,English,
608,Emicida: AmarElo - It's All For Yesterday,89,Documentary,Portuguese,


In [13]:
csv_df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 610 entries, 0 to 609
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   title      610 non-null    object
 1   runtime    610 non-null    int64 
 2   genre      610 non-null    object
 3   languages  610 non-null    object
 4   actors     610 non-null    object
dtypes: int64(1), object(4)
memory usage: 24.0+ KB


# 3. Source2 JSON

In [14]:
json_df2 = pd.read_json('source_data/movie.json')
json_df2.head(1)

,title,rating,year,users_rating,votes,metascore,img_url,countries,languages,actors,genre,tagline,description,directors,runtime,imdb_url
0,A Rainy Day in New York,PG-13,2019.0,6.6,"21,903",44.0,https://m.media-amazon.com/images/M/MV5BODAwZD...,[USA],[English],"[Timothée Chalamet, Elle Fanning, Liev Schreib...","[Comedy, Romance]",None,A young couple arrives in New York for a weeke...,[Woody Allen],92 min,https://www.imdb.com/title/tt7139936/


In [15]:
json_df2.columns

Index(['title', 'rating', 'year', 'users_rating', 'votes', 'metascore',
       'img_url', 'countries', 'languages', 'actors', 'genre', 'tagline',
       'description', 'directors', 'runtime', 'imdb_url'],
      dtype='object')

In [16]:
print(len(csv_df1['title']))
print(len(csv_df1['runtime']))
print(len(json_df2['title']))
print(len(json_df2['runtime']))

610
610
62058
62058


In [17]:
ext_movie_df = json_df2[["title","runtime", "genre","languages","actors"]]
ext_movie_df.head(2)

,title,runtime,genre,languages,actors
0,A Rainy Day in New York,92 min,"[Comedy, Romance]",[English],"[Timothée Chalamet, Elle Fanning, Liev Schreib..."
1,Murder Manual,91 min,"[Horror, Thriller]",[English],"[Emilia Clarke, Hadley Fraser, Sylvia Panacion..."


# 4. Merged CSV and JSON.

In [18]:
# 2 different source is concateneated based on the same columns heads 
movie_df = pd.concat([ext_movie_df,csv_df1], ignore_index=True)
movie_df.head(3)

,title,runtime,genre,languages,actors
0,A Rainy Day in New York,92 min,"[Comedy, Romance]",[English],"[Timothée Chalamet, Elle Fanning, Liev Schreib..."
1,Murder Manual,91 min,"[Horror, Thriller]",[English],"[Emilia Clarke, Hadley Fraser, Sylvia Panacion..."
2,Ferris Bueller's Day Off,103 min,[Comedy],"[English, German]","[Matthew Broderick, Alan Ruck, Mia Sara, Jeffr..."


In [19]:
print(len(movie_df['title']))
print(len(movie_df['runtime']))

62668
62668


In [20]:
# From here unpack json file that has rows as a [] or [[0],[1],[2],...[n]] due to normalize df for RDB
json_df=movie_df

In [21]:
titles_to_list = json_df['title'].tolist()
#titles_to_list
actors_l_of_lists = json_df['actors'].tolist()
actors_l_of_lists
genres_l_of_lists = json_df['genre'].tolist()
genres_l_of_lists
langs_l_of_lists = json_df['languages'].tolist()
runtimes_l_of_lists = json_df['runtime'].tolist()

In [22]:
movie_df[['run_t','mins']] = movie_df.runtime.str.split(expand=True)
movie_df.head(2)

,title,runtime,genre,languages,actors,run_t,mins
0,A Rainy Day in New York,92 min,"[Comedy, Romance]",[English],"[Timothée Chalamet, Elle Fanning, Liev Schreib...",92,min
1,Murder Manual,91 min,"[Horror, Thriller]",[English],"[Emilia Clarke, Hadley Fraser, Sylvia Panacion...",91,min


In [23]:
new_movie_df=movie_df[["title","run_t", "languages", "genre"]]
new_movie_df.head(2)

,title,run_t,languages,genre
0,A Rainy Day in New York,92,[English],"[Comedy, Romance]"
1,Murder Manual,91,[English],"[Horror, Thriller]"


In [24]:
#title & actors normalized tables for each columns 
counter=0
titles_to_expand_match_actors = []
for actors_l in actors_l_of_lists:
    for title in range(0,len(actors_l)):
        titles_to_expand_match_actors.append(titles_to_list[counter])
    counter += 1
#titles_to_expand_match_actors

actors_unpacked_to_outer_list = []
for actors_l in actors_l_of_lists:
    #print(actors_list)
    for actors in actors_l:
        actors_unpacked_to_outer_list.append(actors)
#actors_unpacked_to_outer_list

In [25]:
actors_associated_with_titles_df = pd.DataFrame({'Title': titles_to_expand_match_actors,
                                                 'Performer': actors_unpacked_to_outer_list
                                                }
                                               )
actors_associated_with_titles_df.head()

,Title,Performer
0,A Rainy Day in New York,Timothée Chalamet
1,A Rainy Day in New York,Elle Fanning
2,A Rainy Day in New York,Liev Schreiber
3,A Rainy Day in New York,Suzanne Smith
4,A Rainy Day in New York,Olivia Boreham-Wing


In [26]:
actors_associated_with_titles_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 792624 entries, 0 to 792623
Data columns (total 2 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   Title      792624 non-null  object
 1   Performer  792624 non-null  object
dtypes: object(2)
memory usage: 12.1+ MB


In [27]:
actors_associated_with_titles_df.to_csv('actors1.csv', index = False)
pd.read_csv('actors1.csv')

,Title,Performer
0,A Rainy Day in New York,Timothée Chalamet
1,A Rainy Day in New York,Elle Fanning
2,A Rainy Day in New York,Liev Schreiber
3,A Rainy Day in New York,Suzanne Smith
4,A Rainy Day in New York,Olivia Boreham-Wing
...,...,...
792619,Winter on Fire: Ukraine's Fight for Freedom,
792620,Winter on Fire: Ukraine's Fight for Freedom,
792621,Springsteen on Broadway,
792622,Emicida: AmarElo - It's All For Yesterday,


In [28]:
#title & genres
counter=0
titles_to_expand_match_genres = []
for genre_l in genres_l_of_lists:
    for title in range(0,len(genre_l)):
        titles_to_expand_match_genres.append(titles_to_list[counter])
    counter += 1
# movie titles are expanded to match with different actors played in the movies   
#titles_to_expand_match_genres

genres_unpacked_to_outer_list = []
for genres_l in genres_l_of_lists:
    #print(actors_list)
    for genres in genres_l:
        genres_unpacked_to_outer_list.append(genres)
#genres_unpacked_to_outer_list
#titles_to_expand_match_genres

In [29]:
genres_associated_with_titles_df = pd.DataFrame({'Title': titles_to_expand_match_genres,
                                                 'Genre': genres_unpacked_to_outer_list
                                                }
                                               )
genres_associated_with_titles_df.head(5)

,Title,Genre
0,A Rainy Day in New York,Comedy
1,A Rainy Day in New York,Romance
2,Murder Manual,Horror
3,Murder Manual,Thriller
4,Ferris Bueller's Day Off,Comedy


In [30]:
#genres_associated_with_titles_df.to_csv('genres1.csv', index = False)
# pd.read_csv('genres1.csv')

In [31]:
#title & language
counter=0
titles_to_expand_match_langs = []
for lang_l in langs_l_of_lists:
    for title in range(0,len(lang_l)):
        titles_to_expand_match_langs.append(titles_to_list[counter])
    counter += 1
# movie titles are expanded to match with different actors played in the movies   
titles_to_expand_match_langs

langs_unpacked_to_outer_list = []
for langs_l in langs_l_of_lists:
    #print(actors_list)
    for langs in langs_l:
        langs_unpacked_to_outer_list.append(langs)
#langs_unpacked_to_outer_list

In [32]:
langs_associated_with_titles_df = pd.DataFrame({'Title': titles_to_expand_match_langs,
                                                 'Language': langs_unpacked_to_outer_list
                                                }
                                               )
langs_associated_with_titles_df.head(5)

,Title,Language
0,A Rainy Day in New York,English
1,Murder Manual,English
2,Ferris Bueller's Day Off,English
3,Ferris Bueller's Day Off,German
4,Cape Fear,English


In [33]:
#langs_associated_with_titles_df.to_csv('languages1.csv', index = False)
#pd.read_csv('languages1.csv')

# 5. Cleaning, and preparing tables for ERD 
1. Creating Nflx_movie table PK unique title_id as pk used table with  
   columns such as since Title and run_t are not duplicated to match row vals.

In [34]:
run_t_df = movie_df[["title","run_t"]]
run_t_df

,title,run_t
0,A Rainy Day in New York,92
1,Murder Manual,91
2,Ferris Bueller's Day Off,103
3,Cape Fear,128
4,Mean Girls,97
...,...,...
62663,Winter on Fire: Ukraine's Fight for Freedom,NaN
62664,Winter on Fire: Ukraine's Fight for Freedom,NaN
62665,Springsteen on Broadway,NaN
62666,Emicida: AmarElo - It's All For Yesterday,NaN


# 6. Unique Movie_titles for table, PK movie_id

In [35]:
runtime_table = run_t_df.reset_index().rename(columns = {"index": "movie_id"})
runtime_table

,movie_id,title,run_t
0,0,A Rainy Day in New York,92
1,1,Murder Manual,91
2,2,Ferris Bueller's Day Off,103
3,3,Cape Fear,128
4,4,Mean Girls,97
...,...,...,...
62663,62663,Winter on Fire: Ukraine's Fight for Freedom,NaN
62664,62664,Winter on Fire: Ukraine's Fight for Freedom,NaN
62665,62665,Springsteen on Broadway,NaN
62666,62666,Emicida: AmarElo - It's All For Yesterday,NaN


In [36]:
# on movie title can be many titles 
unique_movie = runtime_table["title"].value_counts()
unique_movie

Dead End                10
Homecoming              10
Vengeance                9
The Outsider             9
The Three Musketeers     9
                        ..
Citizens Band            1
Thaw of the Dead         1
The Last Affair          1
Holy Man                 1
Backlight                1
Name: title, Length: 57148, dtype: int64

In [37]:
# unique_m_title.to_csv('languages1.csv', index = False)

In [38]:
runtime_table = unique_movie.reset_index()
runtime_table.head(2)

,index,title
0,Dead End,10
1,Homecoming,10


In [39]:
movie_table = runtime_table.reset_index().rename(columns ={"level_0":"title_id"})
movie_table.head(2)

,title_id,index,title
0,0,Dead End,10
1,1,Homecoming,10


In [40]:
nflx_movie = movie_table.rename(columns = {"index":"title","title":"numb_of_versions"})
nflx_movie.head(2)

,title_id,title,numb_of_versions
0,0,Dead End,10
1,1,Homecoming,10


In [41]:
nflx_movie = nflx_movie[["title_id","title","numb_of_versions"]]
nflx_movie.head(3)

,title_id,title,numb_of_versions
0,0,Dead End,10
1,1,Homecoming,10
2,2,Vengeance,9


In [42]:
nflx_movie.to_csv('sql_data/nflx_movie.csv1', index = False)

In [43]:
#pd.read_csv('Nflx_movie.csv')

## 7. Table 2  for ERD nflx_m_runtime

In [44]:
run_t_df = movie_df[["title","run_t"]]
run_t_df

,title,run_t
0,A Rainy Day in New York,92
1,Murder Manual,91
2,Ferris Bueller's Day Off,103
3,Cape Fear,128
4,Mean Girls,97
...,...,...
62663,Winter on Fire: Ukraine's Fight for Freedom,NaN
62664,Winter on Fire: Ukraine's Fight for Freedom,NaN
62665,Springsteen on Broadway,NaN
62666,Emicida: AmarElo - It's All For Yesterday,NaN


In [45]:
run_t_clean = run_t_df.fillna(0)
run_t_clean

,title,run_t
0,A Rainy Day in New York,92
1,Murder Manual,91
2,Ferris Bueller's Day Off,103
3,Cape Fear,128
4,Mean Girls,97
...,...,...
62663,Winter on Fire: Ukraine's Fight for Freedom,0
62664,Winter on Fire: Ukraine's Fight for Freedom,0
62665,Springsteen on Broadway,0
62666,Emicida: AmarElo - It's All For Yesterday,0


In [46]:
print(run_t_clean.dtypes)


title    object
run_t    object
dtype: object


In [47]:
runtime_table = run_t_clean.reset_index().rename(columns = {"index": "r_time_id"})
runtime_table

,r_time_id,title,run_t
0,0,A Rainy Day in New York,92
1,1,Murder Manual,91
2,2,Ferris Bueller's Day Off,103
3,3,Cape Fear,128
4,4,Mean Girls,97
...,...,...,...
62663,62663,Winter on Fire: Ukraine's Fight for Freedom,0
62664,62664,Winter on Fire: Ukraine's Fight for Freedom,0
62665,62665,Springsteen on Broadway,0
62666,62666,Emicida: AmarElo - It's All For Yesterday,0


In [48]:
runtime_table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62668 entries, 0 to 62667
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   r_time_id  62668 non-null  int64 
 1   title      62668 non-null  object
 2   run_t      62668 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.4+ MB


In [49]:
#runtime_table["run_t"] = pd.to_numeric(runtime_table["run_t"])
runtime_table.drop(runtime_table.index[18957])

,r_time_id,title,run_t
0,0,A Rainy Day in New York,92
1,1,Murder Manual,91
2,2,Ferris Bueller's Day Off,103
3,3,Cape Fear,128
4,4,Mean Girls,97
...,...,...,...
62663,62663,Winter on Fire: Ukraine's Fight for Freedom,0
62664,62664,Winter on Fire: Ukraine's Fight for Freedom,0
62665,62665,Springsteen on Broadway,0
62666,62666,Emicida: AmarElo - It's All For Yesterday,0


In [50]:
print(runtime_table.dtypes)

r_time_id     int64
title        object
run_t        object
dtype: object


In [51]:
runtime_table['run_t'] = pd.to_numeric(runtime_table['run_t'], errors = 'coerce')
# df['mix_col'] = pd.to_numeric(df['mix_col'], errors='coerce')

In [52]:
runtime_table

,r_time_id,title,run_t
0,0,A Rainy Day in New York,92.0
1,1,Murder Manual,91.0
2,2,Ferris Bueller's Day Off,103.0
3,3,Cape Fear,128.0
4,4,Mean Girls,97.0
...,...,...,...
62663,62663,Winter on Fire: Ukraine's Fight for Freedom,0.0
62664,62664,Winter on Fire: Ukraine's Fight for Freedom,0.0
62665,62665,Springsteen on Broadway,0.0
62666,62666,Emicida: AmarElo - It's All For Yesterday,0.0


In [53]:
print(runtime_table.dtypes)

r_time_id      int64
title         object
run_t        float64
dtype: object


In [54]:
unique_time = runtime_table["run_t"].value_counts()
unique_time

0.0      12775
90.0      3042
85.0      1481
95.0      1430
80.0      1419
         ...  
350.0        1
283.0        1
247.0        1
36.0         1
398.0        1
Name: run_t, Length: 289, dtype: int64

In [55]:
u_runtime_table = unique_time.reset_index()
u_runtime_table.head(5)

,index,run_t
0,0.0,12775
1,90.0,3042
2,85.0,1481
3,95.0,1430
4,80.0,1419


In [56]:
u_rt_table = u_runtime_table.reset_index().rename(columns ={"level_0":"rt_id"})
u_rt_table.head(2)

,rt_id,index,run_t
0,0,0.0,12775
1,1,90.0,3042


In [57]:
nflx_rt = u_rt_table.rename(columns = {"index":"r_time","run_t":"occurrences"})
nflx_rt.head(2)

,rt_id,r_time,occurrences
0,0,0.0,12775
1,1,90.0,3042


In [58]:
nflx_m_runtime = nflx_rt[["rt_id","r_time","occurrences"]]
nflx_m_runtime.head(3)

,rt_id,r_time,occurrences
0,0,0.0,12775
1,1,90.0,3042
2,2,85.0,1481


In [59]:
nflx_m_runtime.to_csv('sql_data/nflx_m_runtime.csv', index = False)

In [60]:
print(nflx_m_runtime.dtypes)

rt_id            int64
r_time         float64
occurrences      int64
dtype: object


# 8. Table 3 nflx_actors for ERD  unque actors

In [61]:
actors_table = actors_associated_with_titles_df
actors_table.head(2)

,Title,Performer
0,A Rainy Day in New York,Timothée Chalamet
1,A Rainy Day in New York,Elle Fanning


In [62]:
# actors_t = run_t_df.reset_index().rename(columns = {"index": "movie_id"})
a_t = actors_table.reset_index()
a_t

,index,Title,Performer
0,0,A Rainy Day in New York,Timothée Chalamet
1,1,A Rainy Day in New York,Elle Fanning
2,2,A Rainy Day in New York,Liev Schreiber
3,3,A Rainy Day in New York,Suzanne Smith
4,4,A Rainy Day in New York,Olivia Boreham-Wing
...,...,...,...
792619,792619,Winter on Fire: Ukraine's Fight for Freedom,
792620,792620,Winter on Fire: Ukraine's Fight for Freedom,
792621,792621,Springsteen on Broadway,
792622,792622,Emicida: AmarElo - It's All For Yesterday,


In [63]:
# actors_t = run_t_df.reset_index().rename(columns = {"index": "movie_id"})
a_t = actors_table.reset_index()
a_t

,index,Title,Performer
0,0,A Rainy Day in New York,Timothée Chalamet
1,1,A Rainy Day in New York,Elle Fanning
2,2,A Rainy Day in New York,Liev Schreiber
3,3,A Rainy Day in New York,Suzanne Smith
4,4,A Rainy Day in New York,Olivia Boreham-Wing
...,...,...,...
792619,792619,Winter on Fire: Ukraine's Fight for Freedom,
792620,792620,Winter on Fire: Ukraine's Fight for Freedom,
792621,792621,Springsteen on Broadway,
792622,792622,Emicida: AmarElo - It's All For Yesterday,


In [64]:
perf_table = a_t.rename(columns= {"index":"actor_id"})
perf_table 

,actor_id,Title,Performer
0,0,A Rainy Day in New York,Timothée Chalamet
1,1,A Rainy Day in New York,Elle Fanning
2,2,A Rainy Day in New York,Liev Schreiber
3,3,A Rainy Day in New York,Suzanne Smith
4,4,A Rainy Day in New York,Olivia Boreham-Wing
...,...,...,...
792619,792619,Winter on Fire: Ukraine's Fight for Freedom,
792620,792620,Winter on Fire: Ukraine's Fight for Freedom,
792621,792621,Springsteen on Broadway,
792622,792622,Emicida: AmarElo - It's All For Yesterday,


In [65]:
uniq_perf = perf_table["Performer"].value_counts()
uniq_perf

                   610
Eric Roberts       256
Charles King       226
Raymond Hatton     216
Roy Barcroft       209
                  ... 
Jennifer Chajon      1
Dan Burman           1
Tony Gomez           1
Whitney Coleman      1
Laura A. Amico       1
Name: Performer, Length: 309660, dtype: int64

In [66]:
to_set_ind = uniq_perf.reset_index()
to_set_ind

,index,Performer
0,,610
1,Eric Roberts,256
2,Charles King,226
3,Raymond Hatton,216
4,Roy Barcroft,209
...,...,...
309655,Jennifer Chajon,1
309656,Dan Burman,1
309657,Tony Gomez,1
309658,Whitney Coleman,1


In [67]:
actor_t = to_set_ind.reset_index().rename(columns ={"level_0":"actor_id"})
actor_t

,actor_id,index,Performer
0,0,,610
1,1,Eric Roberts,256
2,2,Charles King,226
3,3,Raymond Hatton,216
4,4,Roy Barcroft,209
...,...,...,...
309655,309655,Jennifer Chajon,1
309656,309656,Dan Burman,1
309657,309657,Tony Gomez,1
309658,309658,Whitney Coleman,1


In [68]:
nflx_actors_b = actor_t.rename(columns = {"index":"actors","Performer":"appearance"})
nflx_actors_b

,actor_id,actors,appearance
0,0,,610
1,1,Eric Roberts,256
2,2,Charles King,226
3,3,Raymond Hatton,216
4,4,Roy Barcroft,209
...,...,...,...
309655,309655,Jennifer Chajon,1
309656,309656,Dan Burman,1
309657,309657,Tony Gomez,1
309658,309658,Whitney Coleman,1


In [69]:
nflx_actors = nflx_actors_b.drop([0])

In [70]:
nflx_actors.to_csv("sql_data/nflx_actors.csv", index = False)
# nflx_m_runtime.to_csv('sql_data/nflx_m_runtime.csv', index = False)

In [71]:
print(nflx_actors.dtypes)

actor_id       int64
actors        object
appearance     int64
dtype: object


# 9. Table 4. Language table 

In [72]:
language_table = langs_associated_with_titles_df
language_table.head(2)

,Title,Language
0,A Rainy Day in New York,English
1,Murder Manual,English


In [73]:
language_table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75495 entries, 0 to 75494
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Title     75495 non-null  object
 1   Language  75495 non-null  object
dtypes: object(2)
memory usage: 1.2+ MB


In [74]:
l_t = language_table.reset_index()
l_t

,index,Title,Language
0,0,A Rainy Day in New York,English
1,1,Murder Manual,English
2,2,Ferris Bueller's Day Off,English
3,3,Ferris Bueller's Day Off,German
4,4,Cape Fear,English
...,...,...,...
75490,75490,David Attenborough: A Life on Our Planet,g
75491,75491,David Attenborough: A Life on Our Planet,l
75492,75492,David Attenborough: A Life on Our Planet,i
75493,75493,David Attenborough: A Life on Our Planet,s


In [75]:
lang_table = l_t.rename(columns= {"index":"lang_id"})
lang_table

,lang_id,Title,Language
0,0,A Rainy Day in New York,English
1,1,Murder Manual,English
2,2,Ferris Bueller's Day Off,English
3,3,Ferris Bueller's Day Off,German
4,4,Cape Fear,English
...,...,...,...
75490,75490,David Attenborough: A Life on Our Planet,g
75491,75491,David Attenborough: A Life on Our Planet,l
75492,75492,David Attenborough: A Life on Our Planet,i
75493,75493,David Attenborough: A Life on Our Planet,s


In [76]:
uniq_lang = lang_table["Language"].value_counts()
uniq_lang

English        62002
Spanish         1941
French          1515
German           861
Italian          745
               ...  
Acholi             1
y                  1
Bambara            1
Tlingit            1
Kinyarwanda        1
Name: Language, Length: 223, dtype: int64

In [77]:
to_set_idx = uniq_lang.reset_index()
to_set_idx

,index,Language
0,English,62002
1,Spanish,1941
2,French,1515
3,German,861
4,Italian,745
...,...,...
218,Acholi,1
219,y,1
220,Bambara,1
221,Tlingit,1


In [78]:
lang_t = to_set_idx.reset_index().rename(columns ={"level_0":"lang_id"})
lang_t

,lang_id,index,Language
0,0,English,62002
1,1,Spanish,1941
2,2,French,1515
3,3,German,861
4,4,Italian,745
...,...,...,...
218,218,Acholi,1
219,219,y,1
220,220,Bambara,1
221,221,Tlingit,1


In [79]:
nflx_langs = lang_t.rename(columns = {"index":"languages", "Language":"frequency"})
nflx_langs

,lang_id,languages,frequency
0,0,English,62002
1,1,Spanish,1941
2,2,French,1515
3,3,German,861
4,4,Italian,745
...,...,...,...
218,218,Acholi,1
219,219,y,1
220,220,Bambara,1
221,221,Tlingit,1


In [80]:
nflx_languages = nflx_langs[["lang_id","languages","frequency"]]
nflx_languages

,lang_id,languages,frequency
0,0,English,62002
1,1,Spanish,1941
2,2,French,1515
3,3,German,861
4,4,Italian,745
...,...,...,...
218,218,Acholi,1
219,219,y,1
220,220,Bambara,1
221,221,Tlingit,1


In [81]:
nflx_languages.to_csv('sql_data/nflx_languages.csv', index = False)

In [82]:
print(nflx_languages.dtypes)

lang_id       int64
languages    object
frequency     int64
dtype: object


# 10. Table5 nflx_genre for ERD

In [83]:
genre_table = genres_associated_with_titles_df
genre_table.head()

,Title,Genre
0,A Rainy Day in New York,Comedy
1,A Rainy Day in New York,Romance
2,Murder Manual,Horror
3,Murder Manual,Thriller
4,Ferris Bueller's Day Off,Comedy


In [84]:
g_t = genre_table.reset_index()
g_t

,index,Title,Genre
0,0,A Rainy Day in New York,Comedy
1,1,A Rainy Day in New York,Romance
2,2,Murder Manual,Horror
3,3,Murder Manual,Thriller
4,4,Ferris Bueller's Day Off,Comedy
...,...,...,...
141539,141539,David Attenborough: A Life on Our Planet,n
141540,141540,David Attenborough: A Life on Our Planet,t
141541,141541,David Attenborough: A Life on Our Planet,a
141542,141542,David Attenborough: A Life on Our Planet,r


In [85]:
genre_table = g_t.rename(columns= {"index":"genre_id"})
genre_table

,genre_id,Title,Genre
0,0,A Rainy Day in New York,Comedy
1,1,A Rainy Day in New York,Romance
2,2,Murder Manual,Horror
3,3,Murder Manual,Thriller
4,4,Ferris Bueller's Day Off,Comedy
...,...,...,...
141539,141539,David Attenborough: A Life on Our Planet,n
141540,141540,David Attenborough: A Life on Our Planet,t
141541,141541,David Attenborough: A Life on Our Planet,a
141542,141542,David Attenborough: A Life on Our Planet,r


In [86]:
genre_table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 141544 entries, 0 to 141543
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   genre_id  141544 non-null  int64 
 1   Title     141544 non-null  object
 2   Genre     141544 non-null  object
dtypes: int64(1), object(2)
memory usage: 3.2+ MB


In [87]:
uniq_genre = genre_table["Genre"].value_counts()
uniq_genre

Drama        29539
Comedy       19312
Thriller     11785
Romance      11035
Action       10013
             ...  
b                2
Game-Show        1
U                1
Z                1
Talk-Show        1
Name: Genre, Length: 67, dtype: int64

In [88]:
to_set_idx = uniq_genre.reset_index()
to_set_idx

,index,Genre
0,Drama,29539
1,Comedy,19312
2,Thriller,11785
3,Romance,11035
4,Action,10013
...,...,...
62,b,2
63,Game-Show,1
64,U,1
65,Z,1


In [89]:
genre_t = to_set_idx.reset_index().rename(columns ={"level_0":"genre_id"})
genre_t

,genre_id,index,Genre
0,0,Drama,29539
1,1,Comedy,19312
2,2,Thriller,11785
3,3,Romance,11035
4,4,Action,10013
...,...,...,...
62,62,b,2
63,63,Game-Show,1
64,64,U,1
65,65,Z,1


In [90]:
nflx_genres = genre_t.rename(columns = {"index":"genre","Genre":"freq"})
nflx_genres

,genre_id,genre,freq
0,0,Drama,29539
1,1,Comedy,19312
2,2,Thriller,11785
3,3,Romance,11035
4,4,Action,10013
...,...,...,...
62,62,b,2
63,63,Game-Show,1
64,64,U,1
65,65,Z,1


In [91]:
nflx_genres.to_csv("sql_data/nflx_genres.csv", index = False)

In [92]:
print(nflx_genres.dtypes)

genre_id     int64
genre       object
freq         int64
dtype: object


# 11. Table6. Genre Table for ERD 

In [93]:
genre_table = genres_associated_with_titles_df
genre_table


# language_table = langs_associated_with_titles_df
# runtime_table = run_t_df
# runtime_table

,Title,Genre
0,A Rainy Day in New York,Comedy
1,A Rainy Day in New York,Romance
2,Murder Manual,Horror
3,Murder Manual,Thriller
4,Ferris Bueller's Day Off,Comedy
...,...,...
141539,David Attenborough: A Life on Our Planet,n
141540,David Attenborough: A Life on Our Planet,t
141541,David Attenborough: A Life on Our Planet,a
141542,David Attenborough: A Life on Our Planet,r


In [94]:
genre_table.to_csv("sql_data/genre_table1.csv", index = False)

# 12. Table 7 for ERD

In [95]:
language_table = langs_associated_with_titles_df
language_table

,Title,Language
0,A Rainy Day in New York,English
1,Murder Manual,English
2,Ferris Bueller's Day Off,English
3,Ferris Bueller's Day Off,German
4,Cape Fear,English
...,...,...
75490,David Attenborough: A Life on Our Planet,g
75491,David Attenborough: A Life on Our Planet,l
75492,David Attenborough: A Life on Our Planet,i
75493,David Attenborough: A Life on Our Planet,s


In [96]:
language_table.to_csv("sql_data/language_table.csv", index = False)

In [ ]:
# 13. Table8  for ERD

In [97]:
runtime_table

,r_time_id,title,run_t
0,0,A Rainy Day in New York,92.0
1,1,Murder Manual,91.0
2,2,Ferris Bueller's Day Off,103.0
3,3,Cape Fear,128.0
4,4,Mean Girls,97.0
...,...,...,...
62663,62663,Winter on Fire: Ukraine's Fight for Freedom,0.0
62664,62664,Winter on Fire: Ukraine's Fight for Freedom,0.0
62665,62665,Springsteen on Broadway,0.0
62666,62666,Emicida: AmarElo - It's All For Yesterday,0.0


In [98]:
runtime_table_cln = runtime_table.dropna()
runtime_table_cln

,r_time_id,title,run_t
0,0,A Rainy Day in New York,92.0
1,1,Murder Manual,91.0
2,2,Ferris Bueller's Day Off,103.0
3,3,Cape Fear,128.0
4,4,Mean Girls,97.0
...,...,...,...
62663,62663,Winter on Fire: Ukraine's Fight for Freedom,0.0
62664,62664,Winter on Fire: Ukraine's Fight for Freedom,0.0
62665,62665,Springsteen on Broadway,0.0
62666,62666,Emicida: AmarElo - It's All For Yesterday,0.0


In [99]:
runtime_table.to_csv("sql_data/runtime_table_cln.csv", index = False)

In [100]:
#14. Table9 for ERD

In [101]:
actors_table = actors_associated_with_titles_df
actors_table.head(2)

,Title,Performer
0,A Rainy Day in New York,Timothée Chalamet
1,A Rainy Day in New York,Elle Fanning


In [102]:
actors_table.to_csv("sql_data/runtime_table_cln.csv", index = False)

In [103]:
cln_csv = pd.read_csv('sql_data/nflx_movie1.csv')

In [104]:
cln_csv.tail(10)

,title_id,title,numb_of_versions
57138,57138,The Sun Never Sets,1
57139,57139,The Minx,1
57140,57140,Evening Crackling Campfire,1
57141,57141,The Lone Rider Rides On,1
57142,57142,Area 407,1
57143,57143,Holyman Undercover,1
57144,57144,The Yesterday Machine,1
57145,57145,Ladies They Talk About,1
57146,57146,Across the Rio Grande,1
57147,57147,What About Bob?,1


In [105]:
cln_csv.columns

Index(['title_id', 'title', 'numb_of_versions'], dtype='object')

In [106]:
result = cln_csv.iloc[[18262],[1]]
result = cln_csv.iloc[[25427],[1]]
result = cln_csv.iloc[[43625],[1]]
result

,title
43625,"'Sheba, Baby'"


In [107]:
cln_csv.loc[(cln_csv.title == "Feudin', Fussin' and A-Fightin'"),"title"]="Feudin Fussin and A-Fightin"
cln_csv.loc[(cln_csv.title == "God Said, 'Ha!'"),"title"]="God Said. 'Ha!'"
cln_csv.loc[(cln_csv.title == "'Sheba, Baby'"),"title"]="'Sheba Baby'"

cln_csv

,title_id,title,numb_of_versions
0,0,Homecoming,10
1,1,Dead End,10
2,2,Vengeance,9
3,3,The Three Musketeers,9
4,4,The Outsider,9
...,...,...,...
57143,57143,Holyman Undercover,1
57144,57144,The Yesterday Machine,1
57145,57145,Ladies They Talk About,1
57146,57146,Across the Rio Grande,1


In [108]:
cln_csv.to_csv("sql_data/nflx_movie.csv", index = False)


In [110]:
title_id=nflx_movie["title_id"]
title_id

0            0
1            1
2            2
3            3
4            4
         ...  
57143    57143
57144    57144
57145    57145
57146    57146
57147    57147
Name: title_id, Length: 57148, dtype: int64